# Models

> Models Available For This Project

In [ ]:
#| default_exp modeling.models

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from data_system_utilities.file_parsers import yaml

In [ ]:
#| export


from sklearn.model_selection import train_test_split
from sklearn import metrics
from hyperopt import hp
from sklearn.linear_model import LogisticRegression
from rfpimp import * # noqa:

import xgboost as xgb
import numpy as np
import logging
import pandas as pd
import os


logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

This is the DSDE standard process for using Xboost with hyperopt

In [ ]:
#| export


def train_xgb(X_train, X_valid, X_test, y_train, y_valid, y_test,
              evals=20, sub=200000, train=1000000, early_stop=10, verbose=False):
    """
    Binary Classifiation Xgboost DSDE hyper opt approach this should be
    reviewed and customized for your use case.
    """
    parameter_space = {
        'max_depth': hp.choice('max_depth', np.arange(21, dtype=int) + 3),
        'n_estimators': hp.choice('n_estimators', np.arange(301, dtype=int) + 100),
        'gamma': hp.uniform('gamma', 0, 5),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 1, 0.01),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'base_score': hp.quniform('base_score', 0.3, 0.8, 0.01),
        'objective': hp.choice('objective', ['binary:logistic']),
        'eval_metric': hp.choice('eval_metric', ['logloss', 'error', 'auc', 'aucpr', 'map']),
        'use_label_encoder': hp.choice('use_label_encoder', [False]),
        'gpu_id': hp.choice('gpu_id', [-1]),
    }
    logger.info(f'Hyper tuning on {X_train[0:sub].shape[0]} rows')
    opt = HpOptBinary(X_train[0:sub], X_test[0:sub], y_train[0:sub], y_test[0:sub], parameter_space=parameter_space)
    best = opt.optimize(max_evals=evals)
    logger.info(f'Full training on {X_train[0:train].shape[0]} rows')
    model = xgb.XGBClassifier(**best, n_jobs=-1)
    eval_set = [(X_valid, y_valid)]
    model.fit(X_train[0:train], y_train[0:train], eval_set=eval_set, early_stopping_rounds=early_stop, verbose=verbose)
    return model

In [ ]:
#| export


def train_xgb_basic(X_train, X_valid, y_train, y_valid, early_stop=10, verbose=True):
    """
    Binary Classifiation Xgboost Sklearn API Call Basic HyperParameters
    """
    logger.info(f'Training on {X_train.shape[0]} rows')
    model = xgb.XGBClassifier(n_jobs=-1)
    eval_set = [(X_valid, y_valid)]
    model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds=early_stop, verbose=verbose)
    return model

In [ ]:
#| export


def train_logistic(X_train, X_test, y_train, y_test, model=LogisticRegression, evals=20, sub=200000, train=1000000):
    """Logistic Regression Example to show how simple this can be to switch the model being used in this template"""
    parameter_space = {
        'class_weight': hp.choice('class_weight', [None, 'balanced']),
        'warm_start': hp.choice('warm_start', [True, False]),
        'fit_intercept': hp.choice('fit_intercept', [True, False]),
        'tol': hp.uniform('tol', 0.00001, 0.0001),
        'C': hp.uniform('C', 0.05, 3),
        'solver': hp.choice('solver', ['lbfgs']),  # , 'liblinear', 'sag', 'saga']),
        'max_iter': hp.choice('max_iter', range(5, 1000))
    }
    logger.info(f'Hyper tuning on {X_test[0:sub].shape[0]} rows')
    opt = HpOptBinary(X_train[0:sub], X_test[0:sub], y_train[0:sub],
                      y_test[0:sub], model=model, parameter_space=parameter_space)
    best = opt.optimize(max_evals=evals)
    logger.info(f'Full training on {X_test[0:train].shape[0]} rows')
    model = LogisticRegression(**best, n_jobs=-1)
    model.fit(X_train[0:train], y_train[0:train])
    return model

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
os.environ["DATALAKE_SAS_TOKEN_SECRET"]

'?sv=2019-12-12&ss=bfqt&srt=sco&sp=rwdlacupx&se=2031-01-22T06:17:14Z&st=2021-01-21T22:17:14Z&spr=https&sig=kIHogByJjyVWyL6XupA0CBUB1iw12%2FeXWFQiOj5fB5c%3D'